In [9]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2, degrees
import os


In [10]:

# City selection function
def select_cities(group):
    if len(group) <= 10:
        return group
    else:
        # Sort by population (descending) and select 10 cities with bias towards populous ones
        sorted_group = group.sort_values('population', ascending=False)
        if len(sorted_group) > 1:
            indices = np.unique(np.geomspace(1, len(sorted_group), 10).astype(int)) - 1
        else:
            indices = [0]
        return sorted_group.iloc[indices]


In [11]:
# setup 
data_dir = 'data/simple_maps'
results_dir = 'results/'

if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [12]:

# Read the original CSV file
df = pd.read_csv(os.path.join(data_dir, 'uscities.csv'))

# Group by state and apply the selection function
result = df.groupby('state_id').apply(select_cities).reset_index(drop=True)

# Sort the final result by state and population (descending)
result = result.sort_values(['state_id', 'population'], ascending=[True, False])

# Save the selected cities to a new CSV file
result.to_csv(os.path.join(results_dir, 'selected_cities.csv'), index=False)

print(f"Total cities selected: {len(result)}")
print(f"Number of states: {result['state_id'].nunique()}")

# Distance and bearing calculation functions
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth's radius in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance

def calculate_bearing(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    x = sin(dlon) * cos(lat2)
    y = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)
    initial_bearing = atan2(x, y)
    initial_bearing = degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360
    return compass_bearing

# Create a list of all cities with their state and coordinates
cities = result[['city', 'state_id', 'lat', 'lng']].values

# Calculate the distance and bearing matrices
n = len(cities)
distance_matrix_km = np.zeros((n, n))
distance_matrix_miles = np.zeros((n, n))
bearing_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(i+1, n):
        city1, state1, lat1, lon1 = cities[i]
        city2, state2, lat2, lon2 = cities[j]
        distance_km = haversine_distance(lat1, lon1, lat2, lon2)
        distance_miles = distance_km * 0.621371  # Convert km to miles
        bearing = calculate_bearing(lat1, lon1, lat2, lon2)
        distance_matrix_km[i, j] = distance_matrix_km[j, i] = distance_km
        distance_matrix_miles[i, j] = distance_matrix_miles[j, i] = distance_miles
        bearing_matrix[i, j] = bearing
        bearing_matrix[j, i] = (bearing + 180) % 360

# Create city labels with state
city_labels = [f"{city}, {state}" for city, state, _, _ in cities]

# Create DataFrames with the distance and bearing matrices
distance_df_km = pd.DataFrame(distance_matrix_km, index=city_labels, columns=city_labels)
distance_df_miles = pd.DataFrame(distance_matrix_miles, index=city_labels, columns=city_labels)
bearing_df = pd.DataFrame(bearing_matrix, index=city_labels, columns=city_labels)

def export_distance_csv(distance_df_km, distance_df_miles, results_dir):
    distances = []
    for city1 in distance_df_km.index:
        for city2 in distance_df_km.columns:
            if city1 != city2:
                distances.append({
                    'City 1': city1,
                    'City 2': city2, 
                    'Distance (km)': distance_df_km.loc[city1, city2],
                    'Distance (miles)': distance_df_miles.loc[city1, city2]
                })
    
    distance_export_df = pd.DataFrame(distances)
    distance_export_df.to_csv(os.path.join(results_dir, 'city_distances_export.csv'), index=False)
    print("Distances exported to 'city_distances_export.csv'")

def export_bearing_csv(bearing_df, results_dir):
    bearings = []
    for city1 in bearing_df.index:
        for city2 in bearing_df.columns:
            if city1 != city2:
                bearings.append({
                    'City 1': city1,
                    'City 2': city2,
                    'Bearing': bearing_df.loc[city1, city2]
                })
    
    bearing_export_df = pd.DataFrame(bearings)
    bearing_export_df.to_csv(os.path.join(results_dir, 'city_bearings_export.csv'), index=False)
    print("Bearings exported to 'city_bearings_export.csv'")

# Export the data
export_distance_csv(distance_df_km, distance_df_miles, results_dir)
export_bearing_csv(bearing_df, results_dir)

# Print some statistics
print(f"\nNumber of cities: {n}")
print(f"Average distance between cities: {distance_matrix_km.mean():.2f} km ({distance_matrix_miles.mean():.2f} miles)")
print(f"Maximum distance between cities: {distance_matrix_km.max():.2f} km ({distance_matrix_miles.max():.2f} miles)")
print(f"Minimum non-zero distance between cities: {distance_matrix_km[distance_matrix_km > 0].min():.2f} km ({distance_matrix_miles[distance_matrix_miles > 0].min():.2f} miles)")

# Find the city pairs with the most northerly and southerly bearings
max_bearing = bearing_df.max().max()
min_bearing = bearing_df[bearing_df > 0].min().min()

max_bearing_pair = np.where(bearing_matrix == max_bearing)
min_bearing_pair = np.where(bearing_matrix == min_bearing)

print(f"\nMost northerly bearing: {max_bearing:.2f}° from {city_labels[max_bearing_pair[0][0]]} to {city_labels[max_bearing_pair[1][0]]}")
print(f"Most southerly bearing: {min_bearing:.2f}° from {city_labels[min_bearing_pair[0][0]]} to {city_labels[min_bearing_pair[1][0]]}")

/tmp/ipykernel_5303/3918993192.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby('state_id').apply(select_cities).reset_index(drop=True)


Total cities selected: 486
Number of states: 52


Distances exported to 'city_distances_export.csv'
Bearings exported to 'city_bearings_export.csv'

Number of cities: 486
Average distance between cities: 1975.79 km (1227.70 miles)
Maximum distance between cities: 9643.84 km (5992.40 miles)
Minimum non-zero distance between cities: 2.17 km (1.35 miles)

Most northerly bearing: 359.98° from East Middlebury, VT to Highgate Springs, VT
Most southerly bearing: 0.00° from Eastwood, LA to Chanhassen, MN
